## Query Processing Module

#### Query Input

In [1]:
import nltk
# nltk.download('stopwords')
from pyvis.network import Network
import networkx as nx
import pickle
import bs4

%matplotlib inline

DATA="../data/"

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

1.15.0


In [3]:
#Definitions dictionary
import os, json
import pickle
defdict = {}
path_to_json = DATA + 'Definitions/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for i in json_files:
    with open(DATA + 'Definitions/'+i) as json_file:
        data = json.load(json_file)
    defdict.update(data)
definitions = list(defdict.keys())
defvalues = list(defdict.values())

docs = ["Issue and Listing of Non Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client Regulations", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing  of Securities Debt Instruments and Security Receipts", "Delisting of Equity Shares","Issue of Capital And Disclosure Requirements2", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", "Self Regulatory Organisations", "Settlement Proceedings", "Issues and Listing of Muncipal Debt Securities", "Buy Back Of Securities2","Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", "Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]

#Regulations
finalclean48 = []
with open(DATA + 'cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)    
docregs = finalclean48

#Topics documentwise
finalcleantopics48 = []
with open(DATA + 'cleanedregtopics48.pkl','rb') as f:
    finalcleantopics48 = pickle.load(f)    
finaltopics = finalcleantopics48

#vocab definitions
mainvocab = []
with open(DATA + 'mainvocab.pkl','rb') as f:
    mainvocab = pickle.load(f) 

vocabdef = []
with open(DATA + 'vocabdef.pkl','rb') as f:
    vocabdef = pickle.load(f) 

#Additional Documents

with open(DATA + 'concept_filenames.pkl','rb') as f:
    cfile = pickle.load(f) 
with open(DATA + 'concept_sentences.pkl','rb') as f:
    csent = pickle.load(f) 
with open(DATA + 'concept_text.pkl','rb') as f:
    ctext = pickle.load(f) 

with open(DATA + 'informal_filenames.pkl','rb') as f:
    ifile = pickle.load(f) 
with open(DATA + 'informal_sentences.pkl','rb') as f:
    isent = pickle.load(f) 
with open(DATA + 'informal_text.pkl','rb') as f:
    itext = pickle.load(f) 
    
with open(DATA + 'miscvocab.pkl', 'rb') as f:
    miscvocab = pickle.load(f)
    
#Legal Case files
with open(DATA + 'case_filenames.pkl','rb') as f:
    lfile = pickle.load(f) 
with open(DATA + 'case_sentences.pkl','rb') as f:
    lsent = pickle.load(f) 
with open(DATA + 'case_text.pkl','rb') as f:
    ltext = pickle.load(f) 
    
with open(DATA + 'casevocab.pkl', 'rb') as f:
    casevocab = pickle.load(f)

import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords

STOPWORDS = set(
    stopwords.words('english') +\
    ['mm', 'section', 'subsection', 'schedule', '-PRON-', 'chapter', 'regulation', 'repealed', 'thereto','unpublishe', 'thereunder','guideline', 'reference','onus','make','Page','Securities','Exchange','India'])

with open(DATA + 'glossary.json') as f:
    glossary = json.load(f)

In [4]:
def queryvocab(query):
    query1 = ""
    qvocab=[]
    for i in query.split():
        if(i not in stopwords.words()):
            if(i != 'What' and i != 'what' and i!='How'):
                qvocab.append(i)
                query1 = query1 + " " + i
    query = query1
    return qvocab, query

In [5]:
def regextract(docinput, docs, docregs, mainvocab, vocabdef, query, glossary):
    
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    v = []
    
    if(docinput == 'All'):
        rdocs = []
        for i in glossary.keys():
            if(i in query.lower()):
                dval = glossary[i]
                for j in dval:
                    if(j not in rdocs):
                        rdocs.append(j)
        if(len(rdocs)>0):
            docinput = []
            for i in rdocs:
                docinput.append(docs[i])
        else:
            docinput = ['All']
            
    else:
        d = [docinput]
        docinput = d
    
    for dval in docinput:
    
        if(dval != 'All'):
            ind = docs.index(dval)

            rules = docregs[ind]
        
            t=-1
            vocab0 = []
            for i in rules:
                t=t+1
                vocab0.append(i.split())
            
            for i in vocab0:
                for j in i:
                    if(len(j)>2):
                        for m in range(len(symbols)):
                            j = np.char.replace(j, symbols[m], '')
                            j = np.char.replace(j, ",", "")
                            j = np.array2string(j)
                            j = j.replace("'","")
                        if(j not in v):
                            v.append(j)
        else:
            rules = []
            for i in docregs:
                for j in i:
                    rules.append(j)
            vocab0 = mainvocab
        
            for j in vocab0:
                if(len(j)>2):
                    for m in range(len(symbols)):
                        j = np.char.replace(j, symbols[m], '')
                        j = np.char.replace(j, ",", "")
                        j = np.array2string(j)
                        j = j.replace("'","")
                    if(j not in v):
                        v.append(j)
    
    if('All' not in docinput):
        vdef = []
        for i in v:
            if(i in mainvocab and len(i)>2):
                vindex = mainvocab.index(i)
                vdef.append(vocabdef[vindex])
    else:
        vdef = vocabdef
        
    
    return rules, v, vdef, docinput

In [6]:
def querypreprocess(query, qvocab, definitions, finaltopics):
    if('regulations' in qvocab):
        qvocab.remove('regulations')
    if('rules' in qvocab):
        qvocab.remove('rules')
    if('rule' in qvocab):
        qvocab.remove('rule')
    if('chapter' in qvocab):
        qvocab.remove('chapter')
    if('section' in qvocab):
        qvocab.remove('section')
    if('sub' in qvocab):
        qvocab.remove('sub')
    if('SEBI' in qvocab):
        qvocab.remove('SEBI')
    if('means' in qvocab):
        qvocab.remove('means')
    if('shall' in qvocab):
        qvocab.remove('shall')
    if('Securities' in qvocab):
        qvocab.remove('Securities')
    if('Exchange' in qvocab):
        qvocab.remove('Exchange')
    if('pertaining' in qvocab):
        qvocab.remove('pertaining')
    if('India' in qvocab):
        qvocab.remove('India')
        
    query = ""
    for i in qvocab:
        query = query + " " + i
        
    importantwords = []
    for i in qvocab:
        flag = 0
        for j in definitions:
            if(i in j):
                flag = 1
                break
        if(flag == 1):
            importantwords.append(i)

    expansionwords = []
    for i in qvocab:
        flag=0
        for j in finaltopics:
            for k in j:
                if(i == k):
                    flag=1
        if(flag==1):
            if(i not in importantwords):
                importantwords.append(i)
        if(flag==0):
            for j0 in definitions:
                if(i in j0):
                    expansionwords.append(j0)
                    break
    
    qnew = query
    for j in expansionwords:
        k=0
        for i in definitions:
            if(i==j):
                s = i
                s = s + defvalues[k]
                qnew = qnew + ' ' + s
                break
            k=k+1
        else:
            qnew = qnew + ' ' + j
    query = qnew
    
    sent = nlp(query)
    t=0
    for token in sent:
        if(str(token) in importantwords):
            if(t!=0):
                if(str(sent[t-1]) not in importantwords):
                    importantwords.append(str(sent[t-1]))
            if(token.tag_ == 'VB'):
                importantwords.append(str(token))
        t+=1
    
    return query, qvocab, importantwords, expansionwords

## Regulations Retrieval Module

In [7]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = STOPWORDS
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [8]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = stemming(data)
    return data

In [9]:
def tfidfreg(docregs, docinput, category):
    
    if(category == 'regulations'):
        d = [] 
    
        for i in docinput:
            if(i in docs):
                ind = docs.index(i)
                r0 = docregs[ind]
                d.append(r0)
            
        if(len(d)>0):
            docregs = d

    regstring = []
    reverseMap = dict()
    origregstring = [] # flattened docregs
    for ind,i in enumerate(docregs):
        for j in i:
            origregstring.append(j)
            reverseMap[j]=docs[ind]
            data = preprocess(j)
            regstring.append(data)       
    
    
    wordregs = regstring
    
    t=0
    remind = []
    for i in regstring:
        if(i=='\xa0\n' or i == ''):
            remind.append(t)
            regstring.remove(i)
            wordregs.remove(wordregs[t])
        t+=1
        
    origreg = []
    t=0
    while(t<len(origregstring)):
        if(t not in remind):
            origreg.append(origregstring[t])
        t+=1
            
    wordregs = regstring
    
    DF = {}
    wlist = []
    for i in range(len(wordregs)):
        tokens = regstring[i]
        for w in tokens.split():
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
                wlist.append(w)
                
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]
    total_vocab_size = len(DF)

# TFIDF Calculation here
    tf_idf = {}
    N = len(regstring)
    for i in range(N):
        tokens = regstring[i].split()
        counter = Counter(tokens)
        for token in np.unique(tokens):
            if(token in wlist):
                tf = counter[token]/len(tokens)
                df = DF[token]
                idf = np.log((N+1)/(df+1))
                try:
                    tf_idf[token] = tf_idf[token] + tf*idf
                except:
                    tf_idf[token] = tf*idf                
            
    return DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap

In [10]:
def q_vector(qvocab, N, total_vocab, DF):
    Q = np.zeros((len(total_vocab)))
    querytokens = qvocab
    counter = Counter(querytokens)
    words_count = len(querytokens)

    query_weights = {}
    
    for token in np.unique(querytokens):
        if(token in DF):
            tf = counter[token]/words_count
            df = DF[token]
            idf = math.log((N+1)/(df+1))

            try:
                ind = total_vocab.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [11]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [12]:
def cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap):
    d_cosines = []
    
    query_vector = q_vector(qvocab, N, total_vocab, DF)

    t=0
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    indexs=[]
    for i in out:
        flag=0
        for j in qvocab:
            if(j in importantwords):
                flag=flag+2
            else:
                flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    distances=list()
    for ele in answers:
        distances.append(ele[0])
        if ele[1] in reverseMap.keys():
            indexs.append(reverseMap[ele[1]])
        else:
            indexs.append(None)
        
    if(k<len(answers)):
        return answers[0:k],indexs,distances
    else:
        return answers,indexs,distances


In [13]:
from sentence_transformers import SentenceTransformer
from pdb import set_trace
# ;set_trace()
def cosine_similarity_transformer(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap):
    
#     find between query and each origreg 

    d_cosines = []
#     model_name = 'bert-base-nli-mean-tokens'
    model_name='sentence-transformers/roberta-base-nli-stsb-mean-tokens'
    model = SentenceTransformer(model_name)
    query_vector = model.encode(query)
    reg_vectors= model.encode(origreg)
#     from pdb import set_trace;
#     set_trace()    
    t=0
    for r in reg_vectors:
        d_cosines.append(cosine_sim(query_vector, r))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    indexs=[]
    for i in out:
        flag=0
        for j in qvocab:
            if(j in importantwords):
                flag=flag+2
            else:
                flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    distances=list()
    for ele in answers:
        distances.append(ele[0])
        if ele[1] in reverseMap.keys():
            indexs.append(reverseMap[ele[1]])
        else:
            indexs.append(None)
        
    if(k<len(answers)):
        return answers[0:k],indexs,distances
    else:
        return answers,indexs,distances


In [14]:
def vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg,importantwords,reverseMap):
    D = np.zeros((N, len(total_vocab)))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass
    A,indexs,distances = cosine_similarity_transformer(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap)
    
    return A,indexs,distances

In [15]:
# def answermod(A, queryinput):
    
#     ans = []
#     ansreg = []
    
#     t=0
#     while(t<len(A)):
#         A[t][0] = 'title'
#         A[t][1] = [A[t][1]]
#         t+=1
#     t=0
#     df = pd.DataFrame(A,columns=['title','paragraphs'])
    
#     cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
#     cdqa_pipeline.fit_retriever(df=df)
#     print("Predicting")
#     prediction = cdqa_pipeline.predict(queryinput)
#     print("Finished prediction")
#     return prediction[0], prediction[2]

In [16]:
from transformers import pipeline
import json
def answermod(A,queryinput):
    model_name = "deepset/roberta-base-squad2" # roberta QnA model
    model = pipeline(model=model_name, tokenizer=model_name, revision="v1.0", task="question-answering")
    answers=list()
    scores=list()
    regs = [a[1] for a in A]
    for r in regs:
        try:
            result=model(question = queryinput, context= r)
            answers.append(result["answer"])
            scores.append(result["score"])
        except:
            print(r)
            print("=================")
    out = np.array(scores).argsort()[::-1]
    return answers[out[0]], regs[out[0]] , scores[out[0]] # Returns the high score answer,context and the score

In [17]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap = tfidfreg(docregs,docinput,category)
#  Can uncomment below lines to save dump
#         with open(DATA + 'tfidf_regs.pkl','wb') as f:
#             pickle.dump(tf_idf,f)
        print('Extracting relevant answer regulations.......')
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A
    elif(category == 'misc'):
        
        rules = ctext
        rules.extend(itext)
        
        vocab = miscvocab
        N = 440 
        
        with open(DATA + 'misc_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'misc_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'misc_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'misc_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        with open(DATA + 'miscfiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A
        
        sentences = csent
        sentences.extend(isent)
        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1496 
        
        with open(DATA + 'case_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'case_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'case_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'case_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
            
        with open(DATA + 'casefiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        with open(DATA + 'casefiles_sentences_new.pkl','rb') as filer:
            case_sentences = pickle.load(filer)

        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
                
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A

        Anew = []
        for ans in A:
            ind = rules.index(ans[1])
            for sent in case_sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    else:
        return None
        
    print('Extracting answer')
    answer, ansreg ,confidence_score= answermod(A, queryinput)
    print("The confidence score in answer is:\n",confidence_score)
    return answer, ansreg, Aold,indexs,distances



## User input

In [24]:
import time

# queryinput = "What are the legal provisions on appointment and re-appointment of directors of a listed entity?"
# docinput = 'All'
# category = 'regulations'

# queryinput = "I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
# docinput = 'All'
# category = 'legal case'

queryinput = "How much amount will be provided as reward for informants in insider trading cases"
docinput = 'All'
category = 'regulations'

# queryinput = "What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
# docinput = 'All'
# category = 'misc'

k=10
begin = time.time()
answer,ansreg,Aold,indexs,distances = QnAmodel(queryinput,docinput,k, category)
# answer,ansreg = QnAmodel(queryinput,docinput,k, category)

end=time.time()

Preprocessing query......
tf-idf calculation in progress.....
Extracting relevant answer regulations.......
Extracting answer
The confidence score in answer is:
 0.05821599066257477


In [25]:
print(queryinput)
print()
print(answer)
print()
print(ansreg)

How much amount will be provided as reward for informants in insider trading cases

No Reward

7G. No Reward shall be made to an Informant:-


In [26]:
Anew=list()
for x in Aold:
    Anew.append(x[1])
Anew = Anew[:k]
docnames=indexs[:k]
dist=distances[:k]
targets = [i for i in range(len(docnames))]

In [27]:
def resultgraph(targets,answers,doclinks,queryinput,distances):
    idoc="results"
    docnumber = 2
    
    q = 'Results map '
    edge_data = zip(targets, answers,doclinks,distances)
    
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
        
    nt.add_node("questionDot", label='question', title=queryinput, color = "red")
        
    t=0
    igcount=0
    lccount=0
    for e in edge_data:
        dst = e[0]
        h = e[1]
        pdflink = e[2]
        linker = pdflink+'.pdf'
        if pdflink+'.docx' in ifile:
            linker = pdflink+'.docx'
        nt.add_node(str(dst), label=pdflink, title=h, color = "blue", url='./PDFS/'+linker)
        
        nt.add_edge("questionDot", str(dst), value=8, color = "#000000", length=e[3])
        
        t=t+1
    
    nt.show_buttons()  # Allows for adjustments in UI
    nt.show(idoc+".html") # saves it, check for show so that it is automatically opened for user
    
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))
        
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))


In [28]:
resultgraph(targets,Anew,docnames,queryinput,dist)

### Example outputs from model

In [24]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


How much amount will be provided as reward for informants in insider trading cases


The amount of the Reward, if payable


7F.  (1)  Informants  who  are  considered  tentatively  eligible  for  a  Reward,  shall  submit  the Informant Reward Claim Form set out in Schedule E to the Board within the period specified in the intimation sent by the Board.
